# Visualizations

This tutorial illustrates the core visualization utilities available in Ax.

In [1]:
import numpy as np
from ax import (
    Arm,
    ComparisonOp,
    RangeParameter,
    ParameterType,
    SearchSpace, 
    SimpleExperiment, 
    OutcomeConstraint, 
)

from ax.metrics.l2norm import L2NormMetric
from ax.modelbridge.cross_validation import cross_validate
from ax.modelbridge.registry import Models
from ax.plot.contour import interact_contour, plot_contour
from ax.plot.diagnostic import interact_cross_validation
from ax.plot.scatter import(
    interact_fitted,
    plot_objective_vs_constraints,
    tile_fitted,
)
from ax.plot.slice import plot_slice
from ax.utils.measurement.synthetic_functions import hartmann6
from ax.utils.notebook.plotting import render, init_notebook_plotting

init_notebook_plotting()

[INFO 08-09 15:29:56] ipy_plotting: Injecting Plotly library into cell. Do not overwrite or delete cell.


## 1. Create experiment and run optimization

The vizualizations require an experiment object and a model fit on the evaluated data. The routine below is a copy of the Developer API tutorial, so the explanation here is omitted. Retrieving the experiment and model objects for each API paradigm is shown in the respective tutorials

#### 1a. Define search space and evaluation function

In [2]:
noise_sd = 0.5
param_names = [f"x{i+1}" for i in range(6)]  # x1, x2, ..., x6

def noisy_hartmann_evaluation_function(parameterization):
    x = np.array([parameterization.get(p_name) for p_name in param_names])
    noise1, noise2 = np.random.normal(0, noise_sd, 2)

    return {
        "hartmann6": (hartmann6(x) + noise1, noise_sd),
        "l2norm": (np.sqrt((x ** 2).sum()) + noise2, noise_sd)
    }

hartmann_search_space = SearchSpace(
    parameters=[
        RangeParameter(
            name=p_name, parameter_type=ParameterType.FLOAT, lower=0.0, upper=1.0
        )
        for p_name in param_names
    ]
)

In [ ]:
hartmann_search_space

#### 1b. Create Experiment

In [28]:
exp = SimpleExperiment(
    name="test_branin",
    search_space=hartmann_search_space,
    evaluation_function=noisy_hartmann_evaluation_function,
    objective_name="hartmann6",
    minimize=True,
    outcome_constraints=[
        OutcomeConstraint(
            metric=L2NormMetric(
                name="l2norm", param_names=param_names, noise_sd=0.2
            ),
            op=ComparisonOp.LEQ,
            bound=1.25,
            relative=False,
        )
    ],
)
exp.__dict__

{'_status_quo': None,
 '_name': 'test_branin',
 'description': None,
 'runner': None,
 'is_test': False,
 '_data_by_trial': {},
 '_experiment_type': None,
 '_optimization_config': OptimizationConfig(objective=Objective(metric_name="hartmann6", minimize=True), outcome_constraints=[OutcomeConstraint(l2norm <= 1.25)]),
 '_tracking_metrics': {},
 '_time_created': datetime.datetime(2019, 8, 9, 15, 56, 44, 709887),
 '_trials': {},
 '_arms_by_signature': {},
 '_search_space': SearchSpace(parameters=[RangeParameter(name='x1', parameter_type=FLOAT, range=[0.0, 1.0]), RangeParameter(name='x2', parameter_type=FLOAT, range=[0.0, 1.0]), RangeParameter(name='x3', parameter_type=FLOAT, range=[0.0, 1.0]), RangeParameter(name='x4', parameter_type=FLOAT, range=[0.0, 1.0]), RangeParameter(name='x5', parameter_type=FLOAT, range=[0.0, 1.0]), RangeParameter(name='x6', parameter_type=FLOAT, range=[0.0, 1.0])], parameter_constraints=[]),
 '_evaluation_function': <function __main__.noisy_hartmann_evaluation_fu

#### 1c. Run the optimization and fit a GP on all data

After doing (`N_BATCHES=15`) rounds of optimization, fit final GP using all data to feed into the plots.

In [29]:
N_RANDOM = 5
BATCH_SIZE = 3
N_BATCHES = 15

sobol = Models.SOBOL(exp.search_space)
exp.new_batch_trial(generator_run=sobol.gen(N_RANDOM))

for i in range(N_BATCHES):
    print(i)
    intermediate_gp = Models.GPEI(experiment=exp, data=exp.eval())
    print(intermediate_gp.__dict__)
    exp.new_trial(generator_run=intermediate_gp.gen(BATCH_SIZE))
    print(exp.__dict__)

model = Models.GPEI(experiment=exp, data=exp.eval())

0
{'dtype': torch.float64, 'device': device(type='cpu'), '_metric_names': {'l2norm', 'hartmann6'}, '_training_data': [<ax.core.observation.Observation object at 0x0000024D41A07F98>, <ax.core.observation.Observation object at 0x0000024D0003A438>, <ax.core.observation.Observation object at 0x0000024D00035390>, <ax.core.observation.Observation object at 0x0000024D00035F98>, <ax.core.observation.Observation object at 0x0000024D00035DA0>], '_optimization_config': OptimizationConfig(objective=Objective(metric_name="hartmann6", minimize=True), outcome_constraints=[OutcomeConstraint(l2norm <= 1.25)]), '_training_in_design': [True, True, True, True, True], '_status_quo': None, '_arms_by_signature': {'0d81921a8478eb9895724f7f29b86bdd': Arm(name='0_0', parameters={'x1': 0.36395227909088135, 'x2': 0.5726718306541443, 'x3': 0.2803742587566376, 'x4': 0.9008128643035889, 'x5': 0.5376953482627869, 'x6': 0.29502251744270325}), '51a5d237891b29f491ff7eedb11447c5': Arm(name='0_1', parameters={'x1': 0.1805

{'_status_quo': None, '_name': 'test_branin', 'description': None, 'runner': None, 'is_test': False, '_data_by_trial': {0: OrderedDict([(1565341009249, <ax.core.data.Data object at 0x0000024D41A07AC8>)]), 1: OrderedDict([(1565341019749, <ax.core.data.Data object at 0x0000024D424E5898>)])}, '_experiment_type': None, '_optimization_config': OptimizationConfig(objective=Objective(metric_name="hartmann6", minimize=True), outcome_constraints=[OutcomeConstraint(l2norm <= 1.25)]), '_tracking_metrics': {}, '_time_created': datetime.datetime(2019, 8, 9, 15, 56, 44, 709887), '_trials': {0: BatchTrial(experiment_name='test_branin', index=0, status=TrialStatus.COMPLETED), 1: Trial(experiment_name='test_branin', index=1, status=TrialStatus.COMPLETED, arm=Arm(name='1_0', parameters={'x1': 0.581443985051746, 'x2': 0.9793486141195175, 'x3': 0.8899736660720419, 'x4': 0.513091521599078, 'x5': 0.054427098871038304, 'x6': 0.06795519958392383})), 2: Trial(experiment_name='test_branin', index=2, status=Tria

{'_status_quo': None, '_name': 'test_branin', 'description': None, 'runner': None, 'is_test': False, '_data_by_trial': {0: OrderedDict([(1565341009249, <ax.core.data.Data object at 0x0000024D41A07AC8>)]), 1: OrderedDict([(1565341019749, <ax.core.data.Data object at 0x0000024D424E5898>)]), 2: OrderedDict([(1565341030428, <ax.core.data.Data object at 0x0000024D00478CF8>)])}, '_experiment_type': None, '_optimization_config': OptimizationConfig(objective=Objective(metric_name="hartmann6", minimize=True), outcome_constraints=[OutcomeConstraint(l2norm <= 1.25)]), '_tracking_metrics': {}, '_time_created': datetime.datetime(2019, 8, 9, 15, 56, 44, 709887), '_trials': {0: BatchTrial(experiment_name='test_branin', index=0, status=TrialStatus.COMPLETED), 1: Trial(experiment_name='test_branin', index=1, status=TrialStatus.COMPLETED, arm=Arm(name='1_0', parameters={'x1': 0.581443985051746, 'x2': 0.9793486141195175, 'x3': 0.8899736660720419, 'x4': 0.513091521599078, 'x5': 0.054427098871038304, 'x6':

{'_status_quo': None, '_name': 'test_branin', 'description': None, 'runner': None, 'is_test': False, '_data_by_trial': {0: OrderedDict([(1565341009249, <ax.core.data.Data object at 0x0000024D41A07AC8>)]), 1: OrderedDict([(1565341019749, <ax.core.data.Data object at 0x0000024D424E5898>)]), 2: OrderedDict([(1565341030428, <ax.core.data.Data object at 0x0000024D00478CF8>)]), 3: OrderedDict([(1565341046502, <ax.core.data.Data object at 0x0000024D01A5A4E0>)])}, '_experiment_type': None, '_optimization_config': OptimizationConfig(objective=Objective(metric_name="hartmann6", minimize=True), outcome_constraints=[OutcomeConstraint(l2norm <= 1.25)]), '_tracking_metrics': {}, '_time_created': datetime.datetime(2019, 8, 9, 15, 56, 44, 709887), '_trials': {0: BatchTrial(experiment_name='test_branin', index=0, status=TrialStatus.COMPLETED), 1: Trial(experiment_name='test_branin', index=1, status=TrialStatus.COMPLETED, arm=Arm(name='1_0', parameters={'x1': 0.581443985051746, 'x2': 0.9793486141195175,

{'_status_quo': None, '_name': 'test_branin', 'description': None, 'runner': None, 'is_test': False, '_data_by_trial': {0: OrderedDict([(1565341009249, <ax.core.data.Data object at 0x0000024D41A07AC8>)]), 1: OrderedDict([(1565341019749, <ax.core.data.Data object at 0x0000024D424E5898>)]), 2: OrderedDict([(1565341030428, <ax.core.data.Data object at 0x0000024D00478CF8>)]), 3: OrderedDict([(1565341046502, <ax.core.data.Data object at 0x0000024D01A5A4E0>)]), 4: OrderedDict([(1565341062634, <ax.core.data.Data object at 0x0000024D00A3EBA8>)])}, '_experiment_type': None, '_optimization_config': OptimizationConfig(objective=Objective(metric_name="hartmann6", minimize=True), outcome_constraints=[OutcomeConstraint(l2norm <= 1.25)]), '_tracking_metrics': {}, '_time_created': datetime.datetime(2019, 8, 9, 15, 56, 44, 709887), '_trials': {0: BatchTrial(experiment_name='test_branin', index=0, status=TrialStatus.COMPLETED), 1: Trial(experiment_name='test_branin', index=1, status=TrialStatus.COMPLETE

...

{'dtype': torch.float64, 'device': device(type='cpu'), '_metric_names': {'l2norm', 'hartmann6'}, '_training_data': [<ax.core.observation.Observation object at 0x0000024D008FA978>, <ax.core.observation.Observation object at 0x0000024D0199B9E8>, <ax.core.observation.Observation object at 0x0000024D01C5B320>, <ax.core.observation.Observation object at 0x0000024D008CEC50>, <ax.core.observation.Observation object at 0x0000024D008CE4A8>, <ax.core.observation.Observation object at 0x0000024D41992A90>, <ax.core.observation.Observation object at 0x0000024D00A3ED68>, <ax.core.observation.Observation object at 0x0000024D00A3EE10>, <ax.core.observation.Observation object at 0x0000024D00A3E128>, <ax.core.observation.Observation object at 0x0000024D00A3EEB8>, <ax.core.observation.Observation object at 0x0000024D00A3E278>, <ax.core.observation.Observation object at 0x0000024D00478198>, <ax.core.observation.Observation object at 0x0000024D00478F60>, <ax.core.observation.Observation object at 0x0000024

{'dtype': torch.float64, 'device': device(type='cpu'), '_metric_names': {'l2norm', 'hartmann6'}, '_training_data': [<ax.core.observation.Observation object at 0x0000024D008FA2E8>, <ax.core.observation.Observation object at 0x0000024D008FA1D0>, <ax.core.observation.Observation object at 0x0000024D01A704E0>, <ax.core.observation.Observation object at 0x0000024D3F7EA5F8>, <ax.core.observation.Observation object at 0x0000024D3F7EA7F0>, <ax.core.observation.Observation object at 0x0000024D3F7EA7B8>, <ax.core.observation.Observation object at 0x0000024D3F7EAE10>, <ax.core.observation.Observation object at 0x0000024D0003AF98>, <ax.core.observation.Observation object at 0x0000024D01A5AA90>, <ax.core.observation.Observation object at 0x0000024D01A5A358>, <ax.core.observation.Observation object at 0x0000024D01A5A470>, <ax.core.observation.Observation object at 0x0000024D424E5080>, <ax.core.observation.Observation object at 0x0000024D424E5B38>, <ax.core.observation.Observation object at 0x0000024

{'_status_quo': None, '_name': 'test_branin', 'description': None, 'runner': None, 'is_test': False, '_data_by_trial': {0: OrderedDict([(1565341009249, <ax.core.data.Data object at 0x0000024D41A07AC8>)]), 1: OrderedDict([(1565341019749, <ax.core.data.Data object at 0x0000024D424E5898>)]), 2: OrderedDict([(1565341030428, <ax.core.data.Data object at 0x0000024D00478CF8>)]), 3: OrderedDict([(1565341046502, <ax.core.data.Data object at 0x0000024D01A5A4E0>)]), 4: OrderedDict([(1565341062634, <ax.core.data.Data object at 0x0000024D00A3EBA8>)]), 5: OrderedDict([(1565341078817, <ax.core.data.Data object at 0x0000024D00A3E6D8>)]), 6: OrderedDict([(1565341082341, <ax.core.data.Data object at 0x0000024D01A5A630>)]), 7: OrderedDict([(1565341097486, <ax.core.data.Data object at 0x0000024D008FA400>)]), 8: OrderedDict([(1565341115925, <ax.core.data.Data object at 0x0000024D01A70048>)]), 9: OrderedDict([(1565341145280, <ax.core.data.Data object at 0x0000024D0199B828>)]), 10: OrderedDict([(156534116822

{'dtype': torch.float64, 'device': device(type='cpu'), '_metric_names': {'l2norm', 'hartmann6'}, '_training_data': [<ax.core.observation.Observation object at 0x0000024D00035278>, <ax.core.observation.Observation object at 0x0000024D01A5A908>, <ax.core.observation.Observation object at 0x0000024D00A69438>, <ax.core.observation.Observation object at 0x0000024D00A697B8>, <ax.core.observation.Observation object at 0x0000024D00A69FD0>, <ax.core.observation.Observation object at 0x0000024D00A3E978>, <ax.core.observation.Observation object at 0x0000024D00A3E278>, <ax.core.observation.Observation object at 0x0000024D00A3EF60>, <ax.core.observation.Observation object at 0x0000024D00A3EF28>, <ax.core.observation.Observation object at 0x0000024D00A3EEF0>, <ax.core.observation.Observation object at 0x0000024D00A3E9E8>, <ax.core.observation.Observation object at 0x0000024D00A3EC50>, <ax.core.observation.Observation object at 0x0000024D00A3EA20>, <ax.core.observation.Observation object at 0x0000024

{'_status_quo': None, '_name': 'test_branin', 'description': None, 'runner': None, 'is_test': False, '_data_by_trial': {0: OrderedDict([(1565341009249, <ax.core.data.Data object at 0x0000024D41A07AC8>)]), 1: OrderedDict([(1565341019749, <ax.core.data.Data object at 0x0000024D424E5898>)]), 2: OrderedDict([(1565341030428, <ax.core.data.Data object at 0x0000024D00478CF8>)]), 3: OrderedDict([(1565341046502, <ax.core.data.Data object at 0x0000024D01A5A4E0>)]), 4: OrderedDict([(1565341062634, <ax.core.data.Data object at 0x0000024D00A3EBA8>)]), 5: OrderedDict([(1565341078817, <ax.core.data.Data object at 0x0000024D00A3E6D8>)]), 6: OrderedDict([(1565341082341, <ax.core.data.Data object at 0x0000024D01A5A630>)]), 7: OrderedDict([(1565341097486, <ax.core.data.Data object at 0x0000024D008FA400>)]), 8: OrderedDict([(1565341115925, <ax.core.data.Data object at 0x0000024D01A70048>)]), 9: OrderedDict([(1565341145280, <ax.core.data.Data object at 0x0000024D0199B828>)]), 10: OrderedDict([(156534116822

In [34]:
exp.eval()._df

,arm_name,metric_name,mean,sem,trial_index
0,0_0,hartmann6,-0.327065,0.5,0
1,0_0,l2norm,1.706378,0.5,0
2,0_1,hartmann6,0.517767,0.5,0
3,0_1,l2norm,1.541274,0.5,0
4,0_2,hartmann6,-2.112274,0.5,0
5,0_2,l2norm,1.103849,0.5,0
6,0_3,hartmann6,-0.717803,0.5,0
7,0_3,l2norm,1.691051,0.5,0
8,0_4,hartmann6,-0.033152,0.5,0
9,0_4,l2norm,1.534983,0.5,0


In [32]:
exp._trials

{0: BatchTrial(experiment_name='test_branin', index=0, status=TrialStatus.COMPLETED),
 1: Trial(experiment_name='test_branin', index=1, status=TrialStatus.COMPLETED, arm=Arm(name='1_0', parameters={'x1': 0.581443985051746, 'x2': 0.9793486141195175, 'x3': 0.8899736660720419, 'x4': 0.513091521599078, 'x5': 0.054427098871038304, 'x6': 0.06795519958392383})),
 2: Trial(experiment_name='test_branin', index=2, status=TrialStatus.COMPLETED, arm=Arm(name='2_0', parameters={'x1': 0.6542060550587867, 'x2': 0.9692349190067584, 'x3': 0.6495960838628752, 'x4': 0.6688780653118567, 'x5': 0.16057520740656941, 'x6': 0.09112453418536877})),
 3: Trial(experiment_name='test_branin', index=3, status=TrialStatus.COMPLETED, arm=Arm(name='3_0', parameters={'x1': 0.4384896339355276, 'x2': 0.911170710040207, 'x3': 0.9137615477031176, 'x4': 0.6093372385879383, 'x5': 0.16190895811451983, 'x6': 0.23043422756440596})),
 4: Trial(experiment_name='test_branin', index=4, status=TrialStatus.COMPLETED, arm=Arm(name='4_0

## 2. Contour plots

The plot below shows the response surface for `hartmann6` metric as a function of the `x1`, `x2` parameters.

The other parameters are fixed in the middle of their respective ranges, which in this example is 0.5 for all of them.

In [9]:
render(plot_contour(model=model, param_x="x1", param_y="x2", metric_name='hartmann6'))

#### 2a. Interactive contour plot

The plot below allows toggling between different pairs of parameters to view the contours.

In [ ]:
render(interact_contour(model=model, metric_name='hartmann6'))

## 3. Tradeoff plots
This plot illustrates the tradeoffs achievable for 2 different metrics. The plot takes the x-axis metric as input (usually the objective) and allows toggling among all other metrics for the y-axis.

This is useful to get a sense of the pareto frontier (i.e. what is the best objective value achievable for different bounds on the constraint)

In [ ]:
render(plot_objective_vs_constraints(model, 'hartmann6', rel=False))

## 4. Cross-validation plots

CV plots are useful to check how well the model predictions calibrate against the actual measurements. If all points are close to the dashed line, then the model is a good predictor of the real data.

In [ ]:
cv_results = cross_validate(model)
render(interact_cross_validation(cv_results))

## 5. Slice plots

Slice plots show the metric outcome as a function of one parameter while fixing the others. They serve a similar function as contour plots.

In [ ]:
render(plot_slice(model, "x2", "hartmann6"))

## 6. Tile plots

Tile plots are useful for viewing the effect of each arm.

In [ ]:
render(interact_fitted(model, rel=False))